# Подключение библиотек

In [2]:
# Подключение google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [86]:
# Данные
import pandas as pd
import numpy as np
# Векторизация
from sklearn.feature_extraction.text import TfidfVectorizer
# Сжатие матрицы
from sklearn.decomposition import TruncatedSVD
# Поиск ближайших соседей
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator
# Создает пайплайн для обрботки входного сообщения
from sklearn.pipeline import make_pipeline

In [87]:
# Для телеграмм-бота
# ! pip install pytelegrambotapi
import telebot

# Загрузка данных

Данные для обучения можно скачать по ссылке: https://drive.google.com/file/d/1n-CURowM0v-uEX07o5CG2DCvTgTGgem8/view?usp=share_link

In [88]:
# Импорт датасета
df = pd.read_csv('/content/good.tsv', sep = '\t')

# Отбор нужных колонок
df = df[['context_0', 'reply']]

In [89]:
# Размер таблицы
df.shape

(60049, 2)

In [90]:
# Просмотр данных
df.sample(3)

,context_0,reply
42675,он знает все о присяжных и женщинах .,и они все в него влюблены .
52857,что - то не так ?,"нет - нет , совсем ничего ."
49266,"хотите знать , какой я сделал вывод ?",именно это .


# Векторизация

Для подчи данных в модель необходимо их прпобразовать в вектор из чисел

In [91]:
# Инициалцизация алгорима
vectorizer = TfidfVectorizer()
# Обучение алгоритма
vectorizer.fit(df.context_0)
# Получение матрицы от алгоритма
big_matrixs = vectorizer.transform(df.context_0)

In [92]:
# Размер матрицы
big_matrixs.shape

(60049, 14123)

Для уменьшения размерности матрицы признаков следует использовать анализ главных компонентов

In [93]:
# Инициалцизация алгорима
svd = TruncatedSVD(n_components = 300)
# Обучение алгоритма
svd.fit(big_matrixs)
# Получение сжатой матрицы от алгоритма
small_matrixs = svd.transform(big_matrixs)

print(f'Удалось сохранить {round(svd.explained_variance_ratio_.sum(), 2) * 100}% информации')

Удалось сохранить 44.0% информации


In [94]:
# Новый размер матрицы
small_matrixs.shape

(60049, 300)

# Поиск ближайших соседей

In [95]:
def softmax(x):
  """ Функция активации для создания вероятности распределения """
  proba = np.exp(-x)
  return proba / sum(proba)

In [96]:
class NeighborSampler(BaseEstimator):
  """ Класс для поиска ближайшего соседа """
  def __init__ (self, k = 5, temperature = 1.0):
    self.k = k
    self.temperature = temperature

  def fit(self, X, y):
    self.tree_ = BallTree(X)
    self.y_ = np.array(y)

  def predict(self, X, random_sate = None):
    distances, indices = self.tree_.query(X, return_distance = True, k = self.k)
    result = []

    for distance, index in zip(distances, indices):
      result.append(np.random.choice(index, p = softmax(distance * self.temperature)))

    return self.y_[result]

# Оптимизация вывода

Для обработки входного сообщения и предсказания создаим едыный пайплайн

In [97]:
# Создание единого алгоритма
ns = NeighborSampler()
ns.fit(small_matrixs, df.reply)
pipeline = make_pipeline(vectorizer, svd, ns)

# Проверка работоспособности

In [76]:
for _ in range(3):
  massage = input('Вы: ')
  print('Бот:', pipeline.predict([massage])[0])

Вы: здарова дружище
Бот: я почти не слышу .
Вы: как так?
Бот: вот так .
Вы: это печально ...
Бот: да , взгляни - ка .


# Запуск бота

Для создания нового бота необходимо получить токен у бота @BotFather в Телеграмме.

Моего же бота можно найти по логину @erkins_test_bot

In [98]:
TOKEN = '6263029041:AAGDKYK5sk_5OkO9_1Kbk1jn4WaqyA9p3TU'

In [101]:
bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands = ['start', 'help'])
def send_welcome(massage):
  bot.reply_to(massage, 'Привет, поговори со мной')

@bot.message_handler(func = lambda massage : True)
def echo_all(massage):
  bot.reply_to(massage, pipeline.predict([massage.text.lower()])[0])

bot.polling()

Бот работает пока запущенна ячейка, для неограниченного доступа можно запустить его на каком нибудь облачном сервере